# EE 467 Homework 1: Twitter Spam Detection

Welcome to the first homework of EE 467! Here we will apply the ML pipline introduced in the second lab to **Twitter spam message detection problems**. In the past few years, social medias like Twitter has become the target of spam posting. Automatic spam posts can not only affect user experience, but they can also spread misinformation and subtly affect public opinions.

In this homework, we will work on a mini Twitter message dataset extracted from the CRESCI-2017 dataset. This dataset contains 50000 Twitter messages, half of which are sent by real Twitter users and half of which generated by spam bots. Your task is to train classifiers that can tell these two kinds of messages apart. This time, you'll go deeper and try multiple feature extractors and classifiers. Like the labs, you will need to fill out blanks in code cells. You are also going to answer a few questions and **optionally** work on improvements to existing pipeline completely on your own.

## Before You Start

1. You can discuss on Slack if you have questions and want to seek help; however, please try your best to **limit the scope of your question** and **avoid asking directly for answers**. You should also **avoid copy-pasting answers and code** from others.
2. You **may use AI assistants** (e.g., ChatGPT, Gemini, etc.) to support your work. If you do, you must **clearly acknowledge** this by stating:

- **The name of the AI assistant**, and  
- **How it was used** (e.g., brainstorming, clarifying concepts, debugging, editing, generating examples)

You **must not copy and paste** AI-generated content directly into your submission without **fully understanding** what it does and being able to **explain it in your own words**.

3. You can **optionally work on this homework with one other student of this course as a team**, but **each of you needs to submit the homework individually**. If you choose to form a group, please include the name of your teammate here: _(Name of your teammate)_.

## Feature Extraction

We will begin our ML pipeline with feature extraction in this homework, as the dataset is good enough and does not need any special pre-processing. As usual, we will load the dataset into memory, extract samples features into vectors and split the dataset into training and test sets.

In [ ]:
!tar -xf twitter-mini.tar.xz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'


In [ ]:
import pandas as pd

data = pd.read_csv("twitter-mini.csv")
print(data.head(5), "\n")

                                                text  label
0  @baydiangirl it definitely moved some church f...      0
1  @suebob Looks like #TXlege made up new rules f...      0
2  RT @saleemchat: #WJChat #A1B 5 yrs: People ski...      0
3                            @SajidaBalouch Huhhhhh       0
4              @Dukester_94 im the lad of all bibles      0 



In lab 2 we have learned how to use the `CountVectorizer` feature extractor, which counts the occurance of each token (letter here) in a sample (Twitter message text here). This time, we will also try `TfidfVectorizer` and see which feature extractor performs better. Here we will introduce the algorithm it implemented, **TF-IDF**:

> **TF-IDF** is the abbreviation for **term frequency–inverse document frequency**, and is defined as the product of the two frequency. The most classical version of TF-IDF is:
>
> $$
tf-idf(t) = tf(t, d) \cdot idf(t, D) = \frac{f_{t,d}}{|d|} \log \frac{|D|}{|D_t|}
$$
>
> Where $t$ is the term, $D$ is the collection of all documents, $d$ is one of the document that contains $t$ and $D_t$ is the subset of documents that contains term $t$. The advantage of TF-IDF is that frequent and useless words, such as "the", "a" and "an" will receive lower weight, since they appear in almost all documents. On the other hand, unique yet frequent (in any $d \in D_t$) words will get higher weight. Research has shown that TF-IDF is easy to implement and performs relatively well.

Before we run these feature extractors on our dataset, we need to convert each message into a sequence of terms. Since Twitter messages often contain emojis, abbreviations, hashtags and other composition of characters, we can't use the word-level analyzer as we have done in lab 2. Instead, we will generate **character N-grams** from these messages.

> A **character N-gram** consists of N consecutive characters. Here we show all character 1, 2, 3 and 4-grams for word "cold":
> <img src="attachment:char-ngram.jpg" width="50%" />

Both `CountVectorizer` and `TfidfVectorizer` provide support for character N-grams. Refer to `sklearn`'s documents to build a character N-gram analyzer.

In [ ]:
## [ TODO ]
# 1) Extract features of all messages into `feat1` with `CountVectorizer` using CHARACTER trigrams
# 2) Repeat with `TfidfVectorizer` and save generated features as `feat2`
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
print("Extracting features with `CountVectorizer`...")
feat1 = CountVectorizer(analyzer="char", ngram_range=(3, 3)).fit_transform(data["text"])

print("Extracting features with `TfidfVectorizer`...")
feat2 = TfidfVectorizer(analyzer="char", ngram_range=(3, 3)).fit_transform(data["text"])

print("Completed.")

Extracting features with `CountVectorizer`...
Extracting features with `TfidfVectorizer`...
Completed.


We will also try combining two kinds of feature generated above into a single feature. We will use `scipy.sparse.hstack` to concatenate corresonding features for each sample. Remember, concatenated features matrix should have same amount of samples as `feat1` and `feat2`.

In [ ]:
## [ TODO ]
# Concatenate token count features and TF-IDF features on the feature dimension,
# name the combined feature matrix `feat_comb`.
from scipy.sparse import hstack
feat_comb = hstack([feat1, feat2])

print("Completed.")

Completed.


Finally, let's take a look at the labels. In our dataset, 0 represents messages from real Twitter users and 1 represents spam messages.

In [ ]:
y = data["label"].values

## Training

As usual, we will start by doing a 80% / 20% training-test split. Like lab 2, we suggest you to **use the same random seed for all splits** for comparable and reproducible results.

In [ ]:
## [ TODO ]
# 1) Split token count features into 80/20 training and test sets
# 2) Split TF-IDF features into 80/20 training and test sets
# 3) Split combined features into 80/20 training and test sets
from sklearn.model_selection import train_test_split
# Token count features
print("Spliting token count features...")
count_train, count_test, y_train, y_test = train_test_split(feat1, y, test_size=0.2, random_state=42)

# TF-IDF features
print("Spliting TF-IDF features...")
tfidf_train, tfidf_test, y_train, y_test = train_test_split(feat2, y, test_size=0.2, random_state=42)

# Combined features
print("Spliting combined features...")
comb_train, comb_test, y_train, y_test = train_test_split(feat_comb, y, test_size=0.2, random_state=42)

print("Completed.")

Spliting token count features...
Spliting TF-IDF features...
Spliting combined features...
Completed.


We are now ready to train **binary classifiers** for spam message detection. Apart from the `LogisticRegression` classifier you have already seen and used during Lab 1, we will try a few more classifiers:

* `MultinomialNB` (Naive Bayes):  
  This classifier is based on the **Naive Bayes** assumption that, *given the class label*, individual features are conditionally independent.  
  For text classification, `MultinomialNB` is used with **token count** or **TF-IDF-like** features.

  It models the probability of a message belonging to a class $ y \in \{\text{spam}, \text{ham}\} $ as:

  $$
  P(y \mid \mathbf{x}) \propto P(y)\prod_{j=1}^{d} P(x_j \mid y)
   $$

  where $\mathbf{x} = (x_1, x_2, \dots, x_d)$ is the feature vector (e.g., word counts), and $P(x_j \mid y)$ is learned from training data.  
  The predicted label is chosen using:

   $$
  y_{pred} = \arg\max_y \; P(y \mid \mathbf{x})
   $$

  In practice, the computation is done in the **log domain** to avoid numerical underflow:

  $$
  y_{pred} = \arg\max_y \left( \log P(y) + \sum_{j=1}^{d} x_j \log P(x_j \mid y) \right)
  $$

* `LinearSVC`:  
  This classifier is based on **linear SVM**, a popular algorithm for binary classification. The linear SVM algorithm treats the feature vector of each sample as a point in a linear space, and tries to find a hyperplane that separates points of the two classes as well as possible.  

  In practice, we optimize a **squared hinge loss with regularization**, since the features may not be perfectly linearly separable:

   $$
  l_{hinge} = \max(0, y(1 - \mathbf{x}^T \mathbf{w}))^2 + \beta ||\mathbf{w}||_2^2
   $$

  Similarly, the classification result is determined by:

  $$
  y_{pred} = sign(1 - \mathbf{x}^T \mathbf{w})
 $$

We have three kinds of features (**token count**, **TF-IDF**, and **combined**) and three kinds of classifiers. Here we will only try three combinations and compare their performance:

* `MultinomialNB` on token count features  
* `LogisticRegression` on TF-IDF features  
  - Try increasing the number of iterations if a warning of non-convergence pops up  
* `LinearSVC` on combined features  
  - For linear SVM, you can ignore the non-convergence warning as it does not affect performance  


In [ ]:
## [ TODO ]
# 1) Train a `MultinomialNB` classifier called `model1` on token count training set
# 2) Train a `LogisticRegression` classifier called `model2` on TF-IDF training set
# 3) Train a `LinearSVC` classifier called `model_comb` on combined features training set
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
print("Training Naive Bayes classifier on token count training set...")
model1 = MultinomialNB().fit(count_train, y_train)

print("Training logistic regression classifier on TF-IDF training set...")
model2 = LogisticRegression().fit(tfidf_train, y_train)

print("Training SVM classifier on combined features training set...")
model_comb = LinearSVC().fit(comb_train, y_train)

print("Completed.")

Training Naive Bayes classifier on token count training set...
Training logistic regression classifier on TF-IDF training set...
Training SVM classifier on combined features training set...
Completed.


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Evaluation

Finally, it's time to evaluate our models. You probably noticed that we have repeated similar code for different feature extraction and classification models. To avoid that, we will use `for` loops to make evaluation simpler:

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
features = [("Token count", feat1), ("TF-IDF", feat2), ("Combined", feat_comb)]
models = [("Naive Bayes", model1), ("Logistic", model2), ("SVM", model_comb)]

## [ TODO ]
# Complete missing code in the loop body
for i, ((feat_name, feat), (model_name, model)) in enumerate(zip(features, models)):
    # Begin of metrics for combination
    print("====================")
    print(f"{i+1}. {model_name} on {feat_name}:\n")

    # 1) Split features into training and test set
    # (Note: you should use the same random number seed as the one in the training-test split above)
    train, test, y_train, y_test = train_test_split(feat, y, test_size=0.2, random_state=42)

    # 2) Predict labels for training data
    pred_train = model.predict(train)

    # 3) Print metrics for training set (precision, recall and F1 metrics)
    print("Training metrics:")
    precision, recall, fscore, _ = precision_recall_fscore_support(y_train, pred_train, average='binary')
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")
    # Confusion matrix
    print("Training confusion matrix:\n", confusion_matrix(y_train, pred_train), "\n")
    # Accuracy
    print("Train accuracy:", accuracy_score(y_train, pred_train), "\n")

    # 4) Predict labels for test data
    pred_test = model.predict(test)

    # 5) Print metrics for test set (precision, recall and F1 metrics)
    print("Test metrics:")
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, pred_test, average='binary')
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {fscore:.4f}")
    # Confusion matrix
    print("Test Confusion Matrix:\n", confusion_matrix(y_test, pred_test), "\n")
    # Accuracy
    print("Test Accuracy:", accuracy_score(y_test, pred_test))

    # End of metrics for combination
    print("====================\n")

1. Naive Bayes on Token count:

Training metrics:
Precision: 0.9847, Recall: 0.9127, F1-score: 0.9473
Training confusion matrix:
 [[19694   284]
 [ 1748 18274]] 

Train accuracy: 0.9492 

Test metrics:
Precision: 0.9820, Recall: 0.9084, F1-score: 0.9438
Test Confusion Matrix:
 [[4939   83]
 [ 456 4522]] 

Test Accuracy: 0.9461

2. Logistic on TF-IDF:

Training metrics:
Precision: 0.9918, Recall: 0.9387, F1-score: 0.9645
Training confusion matrix:
 [[19822   156]
 [ 1228 18794]] 

Train accuracy: 0.9654 

Test metrics:
Precision: 0.9861, Recall: 0.9265, F1-score: 0.9554
Test Confusion Matrix:
 [[4957   65]
 [ 366 4612]] 

Test Accuracy: 0.9569

3. SVM on Combined:

Training metrics:
Precision: 1.0000, Recall: 0.9986, F1-score: 0.9993
Training confusion matrix:
 [[19978     0]
 [   28 19994]] 

Train accuracy: 0.9993 

Test metrics:
Precision: 0.9488, Recall: 0.9484, F1-score: 0.9486
Test Confusion Matrix:
 [[4767  255]
 [ 257 4721]] 

Test Accuracy: 0.9488



# Questions
Please answer the following questions. You can just write down the answer below each question in this cell.


1. Please compute `tf-idf` of word-level **bigrams** for the following three messages **by hand**. You should use the TF-IDF definition introduced in class.

  * Message 1: "vegetables are good for health"
  * Message 2: "fruits are good source of vitamins"
  * Message 3: "proteins are good for health"

  Compute and sort intermediate and final results for **all occurring bigrams in the following order**:
  
  ```
  ["vegetables are", "are good", "good for", "for health", "fruits are", "good source", "source of", "of vitamins", "proteins are"]
  ```
  
  Then complete the following questions:

  1. Term Frequency (`tf`) of **bigrams for message 2**.
  2. Inverse Document Frequency (`idf`) of **bigrams for message 2**.
  3. `tf-idf` of **bigrams for message 2**.
  4. Is the `tf-idf` value for the term "are good" equal to 0? If it's 0, can you explain why it makes sense to have 0 for that term?
  5. If we consider unigram (1-gram) based bag of words, how many words in these three messages would be assigned a 0 value tf-idf? And please explain why.




1.   TF: [0, 1/5, 0, 0, 1/5, 1/5, 1/5, 1/5, 0]
2.   IDF: [ln(3), 0, ln(3/2), ln(3/2), ln(3), ln(3), ln(3), ln(3), ln(3)]
3.   TF-IDF: [0, 0, 0, 0, 0.2ln(3/2), 0.2ln(3/2), 0.2ln(3), 0.2ln(3), 0]
4.   Yes because "are good" appears in every documents. It's too common and is not important.
5.  "are" and "good" would be assigned a 0 value tf-idf because they had appeared in every document.





2. What does `support` mean in the `classification_report`?
- support means the number of true samples that belongs to that class in the dataset

3. If the training data is highly unbalanced, is `accuracy` a good metric for the classifiers' performance? Does a high accuracy necessarily means a high f1-score? Compare `accuracy` with `f1-score`. Based on your criterion, which type of classifier is better? Please give some justifications to support your answer.
- Accuracy is not a good metric for the classifier. If one class dominates, a classifier can get high accuracy by always predicting the majority class.
- No. f1 is calculated by recall and precision. If a model ignores the minority class, the recall of that class is 0, which lead to f1 be 0; but accuracy can still be high.
- The classifier with higher F1-score is better for imbalanced data. Logistic regression achieves the highest F1-score (0.9554), so it is the best classifier for this task

4. For most cases, the amount of bad data is much less than good data. How can you mitigate the data imbalance issue?
- Data imbalance can be mitigated by oversampling the minority class and undersampling the majority class.

## Model Improvements (Optional; Not Included in Homework Score)

As we have pointed out in the first class, machine learning pipeline does not end with a single iteration of training, testing and evaluation. Here, you will repeat these process with different hyper-parameters and try to improve the spam detection model so that it performs even better.

Below are two **potential improvements** that may be beneficial to the performance. You are encouraged to complete either (or both) of them. Your response to these questions are optional and won't be graded, but we will provide feedbacks to your code and analysis if you choose to answer them. You can add code and text cells below the questions as needed.

1. We use character-level trigrams as tokens during the feature extraction stage, however $N = 3$ is a rather arbitrary choice. What will the performance be like if we use shorter N-grams, such as $N = 1$ and $N = 2$? What about longer N-grams, such as $N = 4$? What if we combine different N-grams, like $N = 1, 2, 3$? Modify the feature extraction code above and play with different (range of) $N$ values. What are **the best (range of) $N$ value on the test set of combined features and SVM classifier**? What metrics did you use to reach this conclusion and why do you think this (range of) $N$ value is the best?

2. During the above training and evaluation process, we are only evaluating three combination of feature extractors and classifiers. Can you try to rewrite the training and evaluation code, such that **every combination** of feature extractors and classifiers can be evaluated? What metrics did you use to reach this conclusion and what is the best combination of feature extractor and classifier in terms of test set performance?

## References

1. CRESCI-2017 Dataset: https://botometer.osome.iu.edu/bot-repository/datasets.html
2. TF-IDF: https://en.wikipedia.org/wiki/Tf%E2%80%93idf
3. N-grams: https://en.wikipedia.org/wiki/N-gram
4. `scikit-learn` documentation: https://scikit-learn.org/stable/modules/classes.html
5. Ridge regression: https://en.wikipedia.org/wiki/Tikhonov_regularization
6. SVM: https://en.wikipedia.org/wiki/Support-vector_machine
7. Lasso: https://en.wikipedia.org/wiki/Lasso_(statistics)
8. ElasticNet: https://en.wikipedia.org/wiki/Elastic_net_regularization
9. Kernel method: https://en.wikipedia.org/wiki/Kernel_method